In [ ]:
import os
import random
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.metrics import confusion_matrix, roc_curve, auc, accuracy_score, precision_score, recall_score, classification_report
from sklearn.utils import class_weight

In [ ]:
SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
os.environ['TF_DETERMINISTIC_OPS'] = '1'
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

In [ ]:
base_dir = "/kaggle/input/brain-tumor-detection"
train_dir = os.path.join(base_dir, "train")
val_dir = os.path.join(base_dir, "val")
test_dir = os.path.join(base_dir, "test")

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.8,1.2],
    zoom_range=0.2,
    shear_range=0.1,
    fill_mode='nearest'
)

val_test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224,224),
    batch_size=16,
    class_mode='binary',
    shuffle=True,
    seed=SEED
)

val_generator = val_test_datagen.flow_from_directory(
    val_dir,
    target_size=(224,224),
    batch_size=16,
    class_mode='binary',
    shuffle=False,
    seed=SEED
)

test_generator = val_test_datagen.flow_from_directory(
    test_dir,
    target_size=(224,224),
    batch_size=16,
    class_mode='binary',
    shuffle=False,
    seed=SEED
)

In [ ]:
train_labels = train_generator.classes
classes = np.unique(train_labels)
cw = class_weight.compute_class_weight('balanced', classes=classes, y=train_labels)
class_weights = dict(enumerate(cw))
print("Class weights:", class_weights)

In [ ]:
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224,224,3))
base_model.trainable = False  
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=output)
model.compile(optimizer=Adam(1e-4), loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
early_stop_1 = EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)
checkpoint_1 = ModelCheckpoint("efficientnetb0_initial.h5", monitor='val_loss', save_best_only=True)
reduce_lr_1 = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)

history_1 = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=25,
    class_weight=class_weights,
    callbacks=[early_stop_1, checkpoint_1, reduce_lr_1],
    verbose=1
)

In [ ]:
base_model.trainable = True


fine_tune_at = len(base_model.layers) - 50  
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

print(f"Fine-tuning son {len(base_model.layers) - fine_tune_at} katmanı...")


model.compile(optimizer=Adam(1e-5/2),  # Daha düşük lr
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
early_stop_2 = EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True)
checkpoint_2 = ModelCheckpoint("efficientnetb0_finetuned.h5", monitor='val_loss', save_best_only=True)
reduce_lr_2 = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=4, verbose=1)

print("=== FINE-TUNING BAŞLIYOR ===")
history_2 = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=20,
    class_weight=class_weights,
    callbacks=[early_stop_2, checkpoint_2, reduce_lr_2],
    verbose=1
)

In [ ]:
plt.figure(figsize=(15,5))

plt.subplot(1,3,1)
plt.plot(history_1.history['accuracy'], label='Train Acc (Initial)', marker='o')
plt.plot(history_1.history['val_accuracy'], label='Val Acc (Initial)', marker='o')
plt.plot(range(len(history_1.history['accuracy']), 
               len(history_1.history['accuracy']) + len(history_2.history['accuracy'])), 
         history_2.history['accuracy'], label='Train Acc (Fine-tune)', marker='s')
plt.plot(range(len(history_1.history['val_accuracy']), 
               len(history_1.history['val_accuracy']) + len(history_2.history['val_accuracy'])), 
         history_2.history['val_accuracy'], label='Val Acc (Fine-tune)', marker='s')
plt.axvline(x=len(history_1.history['accuracy'])-1, color='red', linestyle='--', alpha=0.7, label='Fine-tune Start')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)


plt.subplot(1,3,2)
plt.plot(history_1.history['loss'], label='Train Loss (Initial)', marker='o')
plt.plot(history_1.history['val_loss'], label='Val Loss (Initial)', marker='o')
plt.plot(range(len(history_1.history['loss']), 
               len(history_1.history['loss']) + len(history_2.history['loss'])), 
         history_2.history['loss'], label='Train Loss (Fine-tune)', marker='s')
plt.plot(range(len(history_1.history['val_loss']), 
               len(history_1.history['val_loss']) + len(history_2.history['val_loss'])), 
         history_2.history['val_loss'], label='Val Loss (Fine-tune)', marker='s')
plt.axvline(x=len(history_1.history['loss'])-1, color='red', linestyle='--', alpha=0.7, label='Fine-tune Start')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()


In [ ]:
y_true = test_generator.classes
y_pred_prob = model.predict(test_generator, verbose=0).reshape(-1)
y_pred = (y_pred_prob > 0.5).astype(int)


accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
final_val_loss = history_2.history['val_loss'][-1]

print(f"Accuracy:  {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"Final Val Loss: {final_val_loss:.4f}")


print("\nDetailed Classification Report:")
print(classification_report(y_true, y_pred, target_names=['Normal', 'Tumor']))

In [ ]:
# Confusion Matrix
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=['Normal','Tumor'],
            yticklabels=['Normal','Tumor'])
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
fpr, tpr, thresholds = roc_curve(y_true, y_pred_prob)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(8,6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.3f})')
plt.plot([0,1], [0,1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.grid(True)
plt.show()

print(f"\nFinal AUC Score: {roc_auc:.4f}")

print(f"\n=== MODEL ÖZETİ ===")
print(f"İlk eğitim epoch sayısı: {len(history_1.history['accuracy'])}")
print(f"Fine-tuning epoch sayısı: {len(history_2.history['accuracy'])}")
print(f"Toplam epoch: {len(history_1.history['accuracy']) + len(history_2.history['accuracy'])}")
print(f"En iyi validation accuracy: {max(max(history_1.history['val_accuracy']), max(history_2.history['val_accuracy'])):.4f}")
print(f"Test accuracy: {accuracy:.4f}")
print(f"AUC Score: {roc_auc:.4f}")